<a href="https://colab.research.google.com/github/Jimin980921/Do-it_DeepLearning/blob/master/Do_it_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 그레디언트 업데이트 수식에 패널티 항 반영하기

L1,L2 규제의 강도를 조절하는 매개변수 l1,l2를 추가하고, 기본값은 0

In [0]:
def init(self,learning_rate=0.1,l1=0,l2=0):
  self.w=None
  self.b=None
  self.losses=[]
  self.val_losses=[]
  self.w_history=[]
  self.lr=learning_rate
  self.l1=l1
  self.l2=l2

In [0]:
def fit(self,x,y,epochs=100,x_val=None,y_val=None):
  self.w=np.ones(x.shapes[1])
  self.b=0
  self.w_history.append(self.w.copy())  #가중치 기록
  np.random.seed(42)
  for i in range(epochs):
    loss=0
    indexes=np.random.permutation(np.arrange(len(x)))
    for i in indexes:
      z=self.forpass(x[i])
      a=self.activation(z)
      err=-(y[i]-a)
      w_grad,b_grad=self.backprop(x[i],err)
      #그레디언트에서 패널티 항의 미분값을 더함
      w_grad+=self.l1*np.sign(self.w)+self.l2*self.w
      self.w-=self.lr*w_grad #가중치 업데이트
      self.b-=b_grad

      self.w_history.append(self.w.copy()) #가중치 기록
      a=np.clip(a,1e-10,1-1e-10)
      loss+=-(y[i]*np.log(a)+(1-y[i])*np.log(1-a))
    #에포크마다 평균 손실 기록
    self.losses.append(loss/len(y)+self.reg_loss())
    #검증세트에대한 손실 계산
    self.update_val_loss(x_val,y_val)


# 3.로지스틱 손실 함수 계산에 패널티 항 추가하기

In [0]:
def reg_loss(self):
  return self.l1*np.sum(np.abs(self.w))+self.l2/2*np.sum(self.w**2)

In [0]:
def update_val_loss(self,x_val,y_val):
  if x_val is None:
    return
    val_loss=0
    for i in range(len(x_val)):
      z=self.forpass(x_val[i])
      a=self.activation(z)
      val_loss+=-(y_val[i]*np.log(a)+(1-y_val[i])*np.log(1-a))
    self.val_losses.append(val_loss/len(y_val)+self.reg_loss())

In [0]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
cancer=load_breast_cancer()
x=cancer.data
y=cancer.target
x_train_all,x_test,y_train_all,y_test=train_test_split(x,y,stratify=y,test_size=0.2,random_state=42)

x_train,x_val,y_train,y_val=train_test_split(x_train_all,y_train_all,stratify=y_train_all,test_size=0.2,random_state=42)
#데이터 표준화
train_mean=np.mean(x_train,axis=0)
train_std=np.std(x_train,axis=0)
x_train_scaled=(x_train-train_mean)/train_std

val_mean=np.mean(x_val,axis=0)
val_std=np.std(x_val,axis=0)
x_val_scaled=(x_val-val_mean)/val_std

In [0]:
class SingleLayer:
  def init(self,learning_rate=0.1,l1=0,l2=0):
    self.w=None
    self.b=None
    self.losses=[]
    self.val_losses=[]
    self.w_history=[]
    self.lr=learning_rate
    self.l1=l1
    self.l2=l2
  
  def forpass(self,x):
    z=np.sum(x*self.w)+self.b
    return z
  
  def backprop(self,x,err):
    w_grad=x*err
    b_grad=1*err
    return w_grad,b_grad

  def activation(self,z):
    a=1/(1+np.exp(-z))
    return a
  
  def add_bias(self,x):
    return np.c_[np.ones((x.shape[0],1)),x]  #행렬 맨 앞에 1로 채워진 열 백터를 추가함.

  def fit(self,x,y,epochs=100,x_val=None,y_val=None):
    self.w=np.ones(x.shapes[1])
    self.b=0
    self.w_history.append(self.w.copy())  #가중치 기록
    np.random.seed(42)
    for i in range(epochs):
      loss=0
      indexes=np.random.permutation(np.arrange(len(x)))
      for i in indexes:
        z=self.forpass(x[i])
        a=self.activation(z)
        err=-(y[i]-a)
        w_grad,b_grad=self.backprop(x[i],err)
        #그레디언트에서 패널티 항의 미분값을 더함
        w_grad+=self.l1*np.sign(self.w)+self.l2*self.w
        self.w-=self.lr*w_grad #가중치 업데이트
        self.b-=b_grad

        self.w_history.append(self.w.copy()) #가중치 기록
        a=np.clip(a,1e-10,1-1e-10)
        loss+=-(y[i]*np.log(a)+(1-y[i])*np.log(1-a))
      #에포크마다 평균 손실 기록
      self.losses.append(loss/len(y)+self.reg_loss())
      #검증세트에대한 손실 계산
      self.update_val_loss(x_val,y_val)

  def predict(self,x):
    z=[self.forpass(x_i) for x_i in x]
    return np.array(z)>0

  def score(self,x,y):
    return np.mean(self.predict(x)==y)

  def update_val_loss(self,x_val,y_val):
    if x_val is None:
      return 
      val_loss=0
    for i in range(len(x_val)):
      z=self.forpass(x_val[i])
      a=self.activation(z)
      val_loss+=-(y_val[i]*np.log(a)+(1-y_val[i])*np.log(1-a))
    self.val_losses.append(val_loss/len(y_val)+self.reg_loss())

  def reg_loss(self):
    return self.l1*np.sum(np.abs(self.w))+self.l2/2*np.sum(self.w**2)

# 5.cancer데이터 세트에 L1규제 적용

In [33]:
l1_list=[0.0001,0.001,0.01]

for l1 in l1_list:
  lyr=SingleLayer
  lyr.fit(x_train_scaled,y_train,x_val=x_val_scaled,y_val=y_val)

  plt.plot(lyr.losses)
  plt.plot(lyr.val_losses)
  plt.title('Learning Curve (l1={})'.format(l1))
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend(['train_loss','val_loss'])
  plt.ylim(0,0.3)
  plt.show()

  plt.plot(lyr.w,'bo')
  plt.title('Weight (l1={})'.format(l1))
  plt.xlabel('weight')
  plt.ylabel('value')
  plt.ylim(-4,4)
  plt.show()

TypeError: ignored